![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)

# Token Usage and Cost Monitoring

## Why Token Monitoring Matters

**The Problem:** LLM costs can spiral out of control without proper monitoring.

**Real-World Horror Stories:**
```
Startup A: $50,000 OpenAI bill in first month
Company B: 90% of costs from inefficient context
Team C: 10x cost increase from memory leaks
```

**Why This Matters:**
- 💰 **Budget Control**: Prevent surprise bills
- 📊 **Optimization**: Find inefficiencies
- 🎯 **Planning**: Predict scaling costs
- 🚨 **Alerts**: Catch problems early

## Learning Objectives

You'll learn to:
1. **Track token usage** - Monitor input/output tokens
2. **Calculate costs** - Real-time cost tracking
3. **Set budgets** - Prevent overspending
4. **Analyze patterns** - Find optimization opportunities

## Setup: Simple Token Tracking

Let's build simple functions to track token usage and costs.

In [ ]:
# Simple token usage tracking - no classes needed
import os
from datetime import datetime
from collections import defaultdict
from dotenv import load_dotenv
load_dotenv()

# Global usage tracking (in production, use Redis or database)
usage_stats = {
    'total_input_tokens': 0,
    'total_output_tokens': 0,
    'total_cost': 0.0,
    'requests': 0,
    'daily_usage': defaultdict(lambda: {'tokens': 0, 'cost': 0.0, 'requests': 0})
}

# Current pricing (as of 2024)
PRICING = {
    'gpt-3.5-turbo': {
        'input': 0.0015,   # per 1K tokens
        'output': 0.002    # per 1K tokens
    },
    'gpt-4': {
        'input': 0.03,     # per 1K tokens
        'output': 0.06     # per 1K tokens
    },
    'gpt-4-turbo': {
        'input': 0.01,     # per 1K tokens
        'output': 0.03     # per 1K tokens
    }
}

def count_tokens(text: str) -> int:
    """Simple token counting"""
    try:
        import tiktoken
        encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
        return len(encoding.encode(text))
    except ImportError:
        return len(text) // 4  # Rough approximation

def calculate_cost(input_tokens: int, output_tokens: int, model: str = 'gpt-3.5-turbo') -> float:
    """Calculate cost for a request"""
    if model not in PRICING:
        model = 'gpt-3.5-turbo'  # Default fallback
    
    input_cost = (input_tokens / 1000) * PRICING[model]['input']
    output_cost = (output_tokens / 1000) * PRICING[model]['output']
    
    return input_cost + output_cost

def track_usage(input_text: str, output_text: str, model: str = 'gpt-3.5-turbo'):
    """Track token usage for a request"""
    input_tokens = count_tokens(input_text)
    output_tokens = count_tokens(output_text)
    cost = calculate_cost(input_tokens, output_tokens, model)
    
    # Update global stats
    usage_stats['total_input_tokens'] += input_tokens
    usage_stats['total_output_tokens'] += output_tokens
    usage_stats['total_cost'] += cost
    usage_stats['requests'] += 1
    
    # Update daily stats
    today = datetime.now().strftime('%Y-%m-%d')
    usage_stats['daily_usage'][today]['tokens'] += input_tokens + output_tokens
    usage_stats['daily_usage'][today]['cost'] += cost
    usage_stats['daily_usage'][today]['requests'] += 1
    
    return {
        'input_tokens': input_tokens,
        'output_tokens': output_tokens,
        'total_tokens': input_tokens + output_tokens,
        'cost': cost,
        'model': model
    }

# Test the tracking system
print("💰 Token Usage Tracking System")
print("=" * 40)

# Simulate some requests
sample_requests = [
    ("What machine learning courses are available?", "I found several ML courses: CS301, CS302, and CS401...", 'gpt-3.5-turbo'),
    ("What are the prerequisites for CS301?", "CS301 requires CS101 and CS201 as prerequisites...", 'gpt-3.5-turbo'),
    ("Can you explain neural networks in detail?", "Neural networks are computational models inspired by biological neural networks. They consist of layers of interconnected nodes...", 'gpt-4')
]

for i, (input_text, output_text, model) in enumerate(sample_requests, 1):
    result = track_usage(input_text, output_text, model)
    print(f"Request {i} ({model}):")
    print(f"  Input: {result['input_tokens']} tokens")
    print(f"  Output: {result['output_tokens']} tokens")
    print(f"  Cost: ${result['cost']:.4f}")
    print()

print(f"📊 Total Usage:")
print(f"  Requests: {usage_stats['requests']}")
print(f"  Input tokens: {usage_stats['total_input_tokens']:,}")
print(f"  Output tokens: {usage_stats['total_output_tokens']:,}")
print(f"  Total cost: ${usage_stats['total_cost']:.4f}")

## Concept 1: Cost Analysis and Budgeting

Let's analyze costs and set up simple budgeting.

In [ ]:
# Simple budgeting and cost analysis
def analyze_cost_breakdown():
    """Analyze where costs are coming from"""
    total_tokens = usage_stats['total_input_tokens'] + usage_stats['total_output_tokens']
    
    if total_tokens == 0:
        print("No usage data available")
        return
    
    input_percentage = (usage_stats['total_input_tokens'] / total_tokens) * 100
    output_percentage = (usage_stats['total_output_tokens'] / total_tokens) * 100
    
    avg_tokens_per_request = total_tokens / usage_stats['requests']
    avg_cost_per_request = usage_stats['total_cost'] / usage_stats['requests']
    
    print("📈 Cost Breakdown Analysis:")
    print("=" * 40)
    print(f"Input tokens: {input_percentage:.1f}% of total")
    print(f"Output tokens: {output_percentage:.1f}% of total")
    print(f"Average tokens per request: {avg_tokens_per_request:.0f}")
    print(f"Average cost per request: ${avg_cost_per_request:.4f}")
    
    # Scaling projections
    print(f"\n🚀 Scaling Projections:")
    daily_cost = avg_cost_per_request * 1000  # 1000 requests/day
    monthly_cost = daily_cost * 30
    print(f"1,000 requests/day: ${daily_cost:.2f}/day, ${monthly_cost:.2f}/month")
    
    daily_cost_10k = avg_cost_per_request * 10000  # 10k requests/day
    monthly_cost_10k = daily_cost_10k * 30
    print(f"10,000 requests/day: ${daily_cost_10k:.2f}/day, ${monthly_cost_10k:.2f}/month")

def check_budget(daily_budget: float = 10.0):
    """Simple budget checking"""
    today = datetime.now().strftime('%Y-%m-%d')
    today_usage = usage_stats['daily_usage'][today]
    
    print(f"💳 Budget Check for {today}:")
    print("=" * 40)
    print(f"Daily budget: ${daily_budget:.2f}")
    print(f"Used today: ${today_usage['cost']:.4f}")
    print(f"Remaining: ${daily_budget - today_usage['cost']:.4f}")
    
    usage_percentage = (today_usage['cost'] / daily_budget) * 100
    print(f"Budget used: {usage_percentage:.1f}%")
    
    if usage_percentage > 80:
        print("🚨 WARNING: Over 80% of daily budget used!")
    elif usage_percentage > 50:
        print("⚠️  CAUTION: Over 50% of daily budget used")
    else:
        print("✅ Budget usage is healthy")

def suggest_optimizations():
    """Suggest ways to reduce costs"""
    total_tokens = usage_stats['total_input_tokens'] + usage_stats['total_output_tokens']
    avg_tokens = total_tokens / usage_stats['requests'] if usage_stats['requests'] > 0 else 0
    
    print("💡 Cost Optimization Suggestions:")
    print("=" * 40)
    
    if avg_tokens > 2000:
        print("🔍 HIGH TOKEN USAGE DETECTED:")
        print("   • Implement context compression")
        print("   • Use conversation summarization")
        print("   • Limit conversation history")
        
        # Calculate potential savings
        potential_savings = usage_stats['total_cost'] * 0.3  # 30% reduction
        print(f"   • Potential savings: ${potential_savings:.4f} (30% reduction)")
    
    input_ratio = usage_stats['total_input_tokens'] / total_tokens if total_tokens > 0 else 0
    if input_ratio > 0.8:
        print("📝 HIGH INPUT TOKEN RATIO:")
        print("   • Reduce context size")
        print("   • Remove redundant information")
        print("   • Use more efficient prompts")
    
    print("\n🎯 General Recommendations:")
    print("   • Use GPT-3.5-turbo for simple tasks")
    print("   • Reserve GPT-4 for complex reasoning")
    print("   • Implement caching for repeated queries")
    print("   • Set up usage alerts and budgets")

# Run the analysis
analyze_cost_breakdown()
print()
check_budget(daily_budget=5.0)
print()
suggest_optimizations()

## Concept 2: Usage Patterns and Alerts

Let's build simple monitoring and alerting.

In [ ]:
# Simple usage monitoring and alerts
def monitor_usage_patterns():
    """Analyze usage patterns for insights"""
    print("📊 Usage Pattern Analysis:")
    print("=" * 40)
    
    # Analyze daily usage
    if usage_stats['daily_usage']:
        for date, daily_stats in usage_stats['daily_usage'].items():
            avg_tokens_per_request = daily_stats['tokens'] / daily_stats['requests'] if daily_stats['requests'] > 0 else 0
            avg_cost_per_request = daily_stats['cost'] / daily_stats['requests'] if daily_stats['requests'] > 0 else 0
            
            print(f"Date: {date}")
            print(f"  Requests: {daily_stats['requests']}")
            print(f"  Total tokens: {daily_stats['tokens']:,}")
            print(f"  Total cost: ${daily_stats['cost']:.4f}")
            print(f"  Avg tokens/request: {avg_tokens_per_request:.0f}")
            print(f"  Avg cost/request: ${avg_cost_per_request:.4f}")
            print()
    
    # Identify patterns
    total_requests = usage_stats['requests']
    if total_requests > 0:
        avg_tokens_overall = (usage_stats['total_input_tokens'] + usage_stats['total_output_tokens']) / total_requests
        
        print("🔍 Pattern Insights:")
        if avg_tokens_overall > 1500:
            print("   • High token usage per request - consider compression")
        elif avg_tokens_overall < 500:
            print("   • Efficient token usage - good optimization")
        else:
            print("   • Moderate token usage - room for optimization")

def setup_simple_alerts(cost_threshold: float = 1.0, token_threshold: int = 5000):
    """Simple alerting system"""
    today = datetime.now().strftime('%Y-%m-%d')
    today_usage = usage_stats['daily_usage'][today]
    
    alerts = []
    
    # Cost alerts
    if today_usage['cost'] > cost_threshold:
        alerts.append(f"🚨 COST ALERT: Daily cost ${today_usage['cost']:.4f} exceeds threshold ${cost_threshold:.2f}")
    
    # Token alerts
    if today_usage['tokens'] > token_threshold:
        alerts.append(f"🚨 TOKEN ALERT: Daily tokens {today_usage['tokens']:,} exceeds threshold {token_threshold:,}")
    
    # Request volume alerts
    if today_usage['requests'] > 100:
        alerts.append(f"📈 HIGH VOLUME: {today_usage['requests']} requests today")
    
    print("🔔 Alert System Status:")
    print("=" * 40)
    
    if alerts:
        for alert in alerts:
            print(alert)
    else:
        print("✅ All systems normal - no alerts")
    
    print(f"\n📋 Current Thresholds:")
    print(f"   Daily cost: ${cost_threshold:.2f}")
    print(f"   Daily tokens: {token_threshold:,}")
    print(f"   Request volume: 100")

def generate_usage_report():
    """Generate a simple usage report"""
    print("📄 Usage Report")
    print("=" * 40)
    print(f"Report generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print()
    
    print("📊 Summary Statistics:")
    print(f"   Total requests: {usage_stats['requests']:,}")
    print(f"   Total input tokens: {usage_stats['total_input_tokens']:,}")
    print(f"   Total output tokens: {usage_stats['total_output_tokens']:,}")
    print(f"   Total cost: ${usage_stats['total_cost']:.4f}")
    
    if usage_stats['requests'] > 0:
        avg_cost = usage_stats['total_cost'] / usage_stats['requests']
        total_tokens = usage_stats['total_input_tokens'] + usage_stats['total_output_tokens']
        avg_tokens = total_tokens / usage_stats['requests']
        
        print(f"\n📈 Averages:")
        print(f"   Cost per request: ${avg_cost:.4f}")
        print(f"   Tokens per request: {avg_tokens:.0f}")
        
        # Efficiency metrics
        cost_per_token = usage_stats['total_cost'] / total_tokens if total_tokens > 0 else 0
        print(f"   Cost per token: ${cost_per_token:.6f}")
    
    print(f"\n💡 Recommendations:")
    if usage_stats['total_cost'] > 0.1:
        print("   • Consider implementing context compression")
        print("   • Monitor high-cost requests")
        print("   • Set up automated budgets")
    else:
        print("   • Usage is currently low - good for testing")
        print("   • Prepare optimization strategies for scaling")

# Run monitoring and alerts
monitor_usage_patterns()
setup_simple_alerts(cost_threshold=0.01, token_threshold=1000)
print()
generate_usage_report()